## Word Embedding

For a good tutorial take a look at Tensorflow's original tutorial on this topic: https://www.tensorflow.org/tutorials/word2vec

## Original Lecture Note Link
Motivation and explanation of worc2vec [Stanford NLP Slides](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture2.pdf)

Original Mikolov et. al. papers
- [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf)
- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)

A simple explanation on skip-gram model [blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

On Softmax, hierarchical softmax, negative sampling and NCE
- [On word embeddings - Part 2: Approximating the Softmax](http://ruder.io/word-embeddings-softmax/)
- [Intuition behind NCE (Noise Contrastive Estimation) for word embeddings](https://twitter.com/MShahriariNia/status/908080372298031104): Negative sampling, as the  name suggests, belongs to the family of sampling-based approaches. This family also includes importance sampling and target sampling. Negative sampling is actually a simplified model of an approach called Noise Contrastive Estimation (NCE), e.g. negative sampling makes certain assumption about the number of noise samples to generate (k) and the distribution of noise samples (Q) (negative sampling assumes that kQ(w) = 1) to simplify computation
- [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf):  Mikolov et al.
 have shown training the Skip-gram model that results in faster training and better vector representations for frequent words, compared to more complex hierarchical softmax

## Summary

Traditional way of doing wordembedding would be through __counting__:
- Build a square matrix with dimmensionality of vocabulary where each item is the count. Then do matrix factorization and get an array of doubles for each token. 

Using SVD to find a low rank approximation. Instead of taking the full |V| we take the top k.

This is similar to glove which perdicts matrices itself.

CBOW has a overlapping sliding window of size |w|
1. take dot product of one hot vector to weight matrix to get the embedding of the word
2. calculate the dot product of the word's embedding to all the words to get the distnce to each 
3. to convert distances to probbaility distribution calculate the softmax on top of it
4. if two words are similar teh probability distribution of all the surrounding text would be roughly similar, hence the network training idea

In the basic word2vec we have a softmax, but the summation in softmax over the entire vocabulary is expensive:

- Prob(output|context) = exp(u0 . v_c) / sum_w exp(u_w . v_c)  , where v_c is the context. [Reference](https://arxiv.org/pdf/1410.8251.pdf)

Alternatives:

- instead of full softmax use hierarchical softmax

## Dataset Loader

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import Counter
import random
import os
import sys
sys.path.append('..')
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf

LOG_DIR = './graphs/'

# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/'
EXPECTED_BYTES = 31344016
DATA_FOLDER = 'data/'
FILE_NAME = 'text8.zip'

# $ wc text8
# 0  17005207 100000000 text8
# this means that text8 is a one line file with 17005207 words and 100000000 characters

def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

def download(file_name, expected_bytes):
    """ Download the dataset text8 if it's not already downloaded """
    file_path = DATA_FOLDER + file_name
    if os.path.exists(file_path):
        print("Dataset ready")
        return file_path
    make_dir('data')
    print("Downloading data " + DOWNLOAD_URL + file_name)
    print("Data to be saved in: " + file_path)
    file_name, _ = urllib.request.urlretrieve(DOWNLOAD_URL + file_name, file_path)
    file_stat = os.stat(file_path)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded the file', file_name)
    else:
        raise Exception('File ' + file_name +
                        ' might be corrupted. You should try downloading it with a browser.')
    return file_path

def read_data(file_path):
    """ Read data into a list of tokens 
    There should be 17,005,207 tokens
    """
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split() 
        # tf.compat.as_str() converts the input into the string
    return words

def build_vocab(words, vocab_size):
    """ Build vocabulary of VOCAB_SIZE most frequent words and index of each vocab based on its count"""
    dictionary = dict()
    count = [('UNK', -1)]  # initialize dict of words and their count. 
    
    # Find the k most common words and add them to the list that already has UNK
    print("Calculating most common words") # this takes time
    count.extend(Counter(words).most_common(vocab_size - 1)) 
    index = 0
    make_dir('processed')
    print("For this experiment just use vocab_size 1000.")
    with open(os.path.join(LOG_DIR, 'vocab_1000.tsv'), "w") as f:
        for word, _ in count:
            dictionary[word] = index
            #if index < 1000:
            if index < vocab_size:
                f.write(word + "\n")
            index += 1
    #index_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary#, index_dictionary

def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words] # 0  mean UNK, cool!

# TODO: Check
def generate_sample(index_words, context_window_size):
    """ Form training pairs according to the skip-gram model. (Overlapping slide of variable 
    length window size. 
    yields on all the tokens both before and after) """
    print("generate_sample")
    print(index_words)
    for index, center in enumerate(index_words):
        # for each word in vocabulary generate a random sample around it with length random([0, context_window_size])
        context = random.randint(1, context_window_size)  
        # get a target before the center word
        for target in index_words[max(0, index - context): index]:
            yield center, target
        # get a target after the center wrod
        for target in index_words[index + 1: index + context + 1]:
            yield center, target
        # yields on all the tokens both before and after. Always return a tuple: The center token and the current target 

# TODO: Check
def get_batch(iterator, batch_size):
    """ Group a numerical stream into batches and yield them as Numpy arrays. """
    print("get_batch")
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1])
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)
        yield center_batch, target_batch

def process_data(vocab_size, batch_size, skip_window):
    file_path = download(FILE_NAME, EXPECTED_BYTES)
    words = read_data(file_path)
    #dictionary, _ = build_vocab(words, vocab_size)
    dictionary = build_vocab(words, vocab_size)
    index_words = convert_words_to_index(words, dictionary) # this takes time
    del words # to save memory
    single_gen = generate_sample(index_words, skip_window)
    return get_batch(single_gen, batch_size)

#def get_index_vocab(vocab_size):
#    file_path = download(FILE_NAME, EXPECTED_BYTES)
#    words = read_data(file_path)
#    return build_vocab(words, vocab_size)


## Imports and Constants

In [2]:
# imports and constants

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

VOCAB_SIZE = 10000
BATCH_SIZE = 128
EMBED_SIZE = 128 # dimension of the word embedding vectors
SKIP_WINDOW = 1 # the context window
NUM_SAMPLED = 64    # Number of negative examples to sample.
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
SKIP_STEP = 2000 # how many steps to skip before reporting the loss



## Load Data

In [3]:
batch_gen = process_data(VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW)

Dataset ready
Calculating most common words
For this experiment just use vocab_size 1000.


## Training

In [4]:
def word2vec(batch_gen):
    """ Build the graph for word2vec model and train it """
    # Step 1: define the placeholders for input and output
    # center_words have to be int to work on embedding lookup

    with tf.name_scope('data'):
        X = tf.placeholder(tf.int32, [BATCH_SIZE], name="X_CenterWords")  # input word's index
        Y = tf.placeholder(tf.int32, [BATCH_SIZE,1], name="Y_TargetWords")  # output word's index. note: [,1]

    # Step 2: define weights. In word2vec, it's actually the weights that we care about
    # vocab size x embed size
    # initialized to random uniform -1 to 1
    with tf.name_scope('embedding_matrix'):
        embed_matrix = tf.Variable(tf.random_uniform([VOCAB_SIZE, EMBED_SIZE],minval=-1, maxval=1), name="embed_matrix") # used for lookup via index

    # Step 3: define the inference
    # get the embed of input words using tf.nn.embedding_lookup
    # embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embed')
    with tf.name_scope('loss'):
        embed = tf.nn.embedding_lookup(embed_matrix, X, name='embed')

        # Step 4: construct variables for NCE loss
        # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
        # nce_weight (vocab size x embed size), intialized to truncated_normal stddev=1.0 / (EMBED_SIZE ** 0.5)
        # bias: vocab size, initialized to 0
        nce_weight = tf.Variable(tf.truncated_normal([VOCAB_SIZE, EMBED_SIZE], stddev=1.0 / (EMBED_SIZE ** 0.5)), name="nce_weight")
        nce_bias = tf.Variable(tf.zeros([VOCAB_SIZE]), name="nce_bias")

        # define loss function to be NCE loss function
        # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
        # need to get the mean accross the batch
        # note: you should use embedding of center words for inputs, not center words themselves
        #
        # nce_loss params:
        #     labels: A `Tensor` of type `int64` and shape `[batch_size, num_true]`. The target classes.  Note that this format differs from
        #             the `labels` argument of `nn.softmax_cross_entropy_with_logits`.
        #     inputs: A `Tensor` of shape `[batch_size, dim]`.  The forward activations of the input network.
        #     num_sampled: An `int`.  The number of classes to randomly sample per batch.
        #     num_classes: An `int`. The number of possible classes.
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                             biases=nce_bias, 
                                             labels=Y, # index of the target word
                                             inputs=embed,      # input embedding
                                             num_sampled=NUM_SAMPLED, # 64 negative samples
                                             num_classes=VOCAB_SIZE   # num_classes=VOCAB_SIZE 
                                            ), 
                                             name='loss')
        
    # Step 5: define optimizer
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

    with tf.Session() as sess:
        # initialize variables
        sess.run(tf.global_variables_initializer())

        total_loss = 0.0 # we use this to calculate the average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
        for index in range(NUM_TRAIN_STEPS):
            centers, targets = next(batch_gen)
            
            loss_batch, _ = sess.run([loss, optimizer], 
                                     feed_dict={X: centers, Y: targets})

            total_loss += loss_batch
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                total_loss = 0.0
        writer.close()
        
        ##
        # save the model for visualizing embedidngs
        saver = tf.train.Saver()
        saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), NUM_TRAIN_STEPS)
        
        # Add meta data as labels for embeddings
        from tensorflow.contrib.tensorboard.plugins import projector
        # Use the same LOG_DIR where you stored your checkpoint.
        summary_writer = tf.summary.FileWriter(LOG_DIR)

        # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
        config = projector.ProjectorConfig()

        # You can add multiple embeddings. Here we add only one.
        embedding = config.embeddings.add()
        embedding.tensor_name = embed_matrix.name
        # Link this tensor to its metadata file (e.g. labels).
        embedding.metadata_path = 'vocab_1000.tsv'

        # Saves a configuration file that TensorBoard will read during startup.
        projector.visualize_embeddings(summary_writer, config)



word2vec(batch_gen)

get_batch
generate_sample


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Average loss at step 1999:  41.2
Average loss at step 3999:   8.4
Average loss at step 5999:   5.8
Average loss at step 7999:   5.1
Average loss at step 9999:   4.8
Average loss at step 11999:   4.7
Average loss at step 13999:   4.6
Average loss at step 15999:   4.5
Average loss at step 17999:   4.4
Average loss at step 19999:   4.6
Average loss at step 21999:   4.5
Average loss at step 23999:   4.5
Average loss at step 25999:   4.5
Average loss at step 27999:   4.5
Average loss at step 29999:   4.4
Average loss at step 31999:   4.5
Average loss at step 33999:   4.5
Average loss at step 35999:   4.5
Average loss at step 37999:   4.4
Average loss at step 39999:   4.4
Average loss at step 41999:   4.4
Average loss at step 43999:   4.4
Average loss at step 45999:   4.5
Average loss at step 47999:   4.4
Average loss at step 49999:   4.4
Average loss at step 51999:   4.4
Average loss at step 53999:   4.4
Average loss at step 55999:   4.4
Average loss at step 57999:   4.5
Average loss at ste